## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Port Elizabeth,-33.9180,25.5701,63.10,94,90,10.36,ZA,2021-12-04 22:29:52,light rain
1,1,Vaini,-21.2000,-175.2000,84.36,74,40,12.66,TO,2021-12-04 22:28:14,light rain
2,2,Dudinka,69.4058,86.1778,-30.39,100,92,5.57,RU,2021-12-04 22:35:05,overcast clouds
3,3,Kodiak,57.7900,-152.4072,26.11,68,90,8.05,US,2021-12-04 22:28:28,overcast clouds
4,4,Rikitea,-23.1203,-134.9692,76.06,71,40,10.00,PF,2021-12-04 22:23:06,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum temperature requirement for your destination?"))
max_temp = float(input("What is your maximum temperature requirement for your destination?"))

What is your minimum temperature requirement for your destination?70
What is your maximum temperature requirement for your destination?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.TYer
Temp_Filter_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
Temp_Filter_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
1,1,Vaini,-21.2000,-175.2000,84.36,74,40,12.66,TO,2021-12-04 22:28:14,light rain
4,4,Rikitea,-23.1203,-134.9692,76.06,71,40,10.00,PF,2021-12-04 22:23:06,scattered clouds
9,9,Pala,9.3642,14.9046,78.04,28,88,6.82,TD,2021-12-04 22:35:05,overcast clouds
11,11,Saint-Philippe,-21.3585,55.7679,76.55,83,29,9.28,RE,2021-12-04 22:31:30,scattered clouds
13,13,Georgetown,5.4112,100.3354,78.89,84,20,9.22,MY,2021-12-04 22:25:24,few clouds
14,14,Kapaa,22.0752,-159.3190,72.12,68,75,1.01,US,2021-12-04 22:28:21,broken clouds
15,15,Butaritari,3.0707,172.7902,83.21,78,28,19.98,KI,2021-12-04 22:33:54,scattered clouds
16,16,Sao Felix Do Xingu,-6.6447,-51.9950,73.62,96,100,1.41,BR,2021-12-04 22:35:06,overcast clouds
18,18,Mocajuba,-2.5842,-49.5072,75.87,93,100,0.45,BR,2021-12-04 22:35:07,overcast clouds
20,20,Mahebourg,-20.4081,57.7000,77.29,78,20,5.75,MU,2021-12-04 22:29:13,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
Temp_Filter_df.count()

City_ID                255
City                   255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Country                254
Date                   255
Current Description    255
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Temp_Filter_df.dropna()
Temp_Filter_df.count()

City_ID                255
City                   255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Country                254
Date                   255
Current Description    255
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Temp_Filter_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,84.36,light rain,-21.2000,-175.2000,
4,Rikitea,PF,76.06,scattered clouds,-23.1203,-134.9692,
9,Pala,TD,78.04,overcast clouds,9.3642,14.9046,
11,Saint-Philippe,RE,76.55,scattered clouds,-21.3585,55.7679,
13,Georgetown,MY,78.89,few clouds,5.4112,100.3354,
14,Kapaa,US,72.12,broken clouds,22.0752,-159.3190,
15,Butaritari,KI,83.21,scattered clouds,3.0707,172.7902,
16,Sao Felix Do Xingu,BR,73.62,overcast clouds,-6.6447,-51.9950,
18,Mocajuba,BR,75.87,overcast clouds,-2.5842,-49.5072,
20,Mahebourg,MU,77.29,few clouds,-20.4081,57.7000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("hotel not found")
        

hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,84.36,light rain,-21.2000,-175.2000,Keleti Beach Resort
4,Rikitea,PF,76.06,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Pala,TD,78.04,overcast clouds,9.3642,14.9046,Nectar Hôtel de Pala
11,Saint-Philippe,RE,76.55,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Georgetown,MY,78.89,few clouds,5.4112,100.3354,Cititel Penang


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Max Temp} °F {Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))